请用tsne生成类似链接中的图片：https://user-images.githubusercontent.com/39433014/222371713-92153612-2d04-409b-bee8-7a50afed1e56.png。我已经通过mmseg中的方法获取了cityscapes数据集的特征图和标签，其中特征图的shape(200,19,1024,2048),200表示图片数量，19表示通道数，1024表示高度，2048表示宽度，其中，每个值是浮点数。标签的shape是（200，1024，2048）,200表示数量，1024表示高度，2048表示宽度，其中每个值代表了像素的类别，范围是0到18。如何用tsne可视化语义分割特征图的特征分布情况呢？其中，属于不同类的特征用不同的颜色表示。

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Load feature maps and labels
features = np.load('features.npy')  # shape: (200, 19, 1024, 2048)
labels = np.load('labels.npy')  # shape: (200, 1024, 2048)

# Reshape feature maps to 2D array
features_2d = features.reshape((-1, 19))

# Reshape labels to 1D array
labels_1d = labels.reshape((-1,))

# Define colors for each class
colors = ['black', 'white', 'red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'gray', 'brown', 'orange', 'pink', 'purple', 'olive', 'lime', 'teal', 'navy', 'maroon', 'gold']

# Create a TSNE object and fit the feature maps
from MulticoreTSNE import MulticoreTSNE

tsne = MulticoreTSNE(n_components=2, perplexity=10, learning_rate=100, n_jobs=4)
features_tsne = tsne.fit_transform(features_2d)



# Plot the feature maps with different colors for each class
fig, ax = plt.subplots(figsize=(10, 10))
for i in range(19):
    indices = np.where(labels_1d == i)[0]
    ax.scatter(features_tsne[indices, 0], features_tsne[indices, 1], c=colors[i], label=str(i))
ax.legend()
plt.show()


下采样一下，我发现(1024,2048)维度太大了，我想将特征图和标签都进行下采样到(256,512)

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from skimage.transform import resize

# Load feature maps and labels
features = np.load('features.npy')  # shape: (200, 19, 1024, 2048)
labels = np.load('labels.npy')  # shape: (200, 1024, 2048)

# Downsample feature maps to (256, 512)
features_downsampled = np.zeros((200, 19, 256, 512))
for i in range(200):
    for j in range(19):
        features_downsampled[i, j] = resize(features[i, j], (256, 512))

# Downsample labels to (256, 512)
labels_downsampled = np.zeros((200, 256, 512))
for i in range(200):
    labels_downsampled[i] = resize(labels[i], (256, 512))

# Reshape feature maps to 2D array
features_2d = features_downsampled.reshape((-1, 19))

# Reshape labels to 1D array
labels_1d = labels_downsampled.reshape((-1,))

# Define colors for each class
colors = ['black', 'white', 'red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'gray', 'brown', 'orange', 'pink', 'purple', 'olive', 'lime', 'teal', 'navy', 'maroon', 'gold']

# Create a TSNE object and fit the feature maps
from MulticoreTSNE import MulticoreTSNE

tsne = MulticoreTSNE(n_components=2, perplexity=10, learning_rate=100, n_jobs=4)
features_tsne = tsne.fit_transform(features_2d)

# Plot the feature maps with different colors for each class
fig, ax = plt.subplots(figsize=(10, 10))
for i in range(19


我发现(1024,2048)维度太大了，我想通过卷积的方式将特征图和标签都进行下采样到(32,64)，其中值的范围不要发生变化

In [ ]:
import numpy as np
from skimage.measure import block_reduce
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Load feature maps and labels
features = np.load('features.npy')  # shape: (200, 19, 1024, 2048)
labels = np.load('labels.npy')  # shape: (200, 1024, 2048)

# Downsample feature maps and labels
features_downsampled = block_reduce(features, block_size=(1, 1, 32, 64), func=np.mean)
labels_downsampled = block_reduce(labels, block_size=(1, 32, 64), func=np.mean)

# Reshape feature maps to 2D array
features_2d = features_downsampled.reshape((-1, 19))

# Reshape labels to 1D array
labels_1d = labels_downsampled.reshape((-1,))

# Define colors for each class
colors = ['black', 'white', 'red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'gray', 'brown', 'orange', 'pink', 'purple', 'olive', 'lime', 'teal', 'navy', 'maroon', 'gold']

# Create a TSNE object and fit the feature maps
from MulticoreTSNE import MulticoreTSNE

tsne = MulticoreTSNE(n_components=2, perplexity=10, learning_rate=100, n_jobs=4)
features_tsne = tsne.fit_transform(features_2d)

# Plot the feature maps with different colors for each class
fig, ax = plt.subplots(figsize=(10, 10))
for i in range(19):
    indices = np.where(labels_1d == i)[0]
    ax.scatter(features_tsne[indices, 0], features_tsne[indices, 1], c=colors[i], label=str(i))
ax.legend()

# Save the plot as a PNG image


使用skimage.measure.block_reduce进行下采样时，报错，NameError: name 'block_size' is not defined